<a href="https://colab.research.google.com/github/eisbetterthanpi/JEPA/blob/main/JEPA_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### setup

In [ ]:
# https://openreview.net/pdf?id=BZ5a1r-kVsf
# import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import collections
device = "cuda" if torch.cuda.is_available() else "cpu"
!pip install optuna
import optuna


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 27.3 MB/s 
     |████████████████████████████████| 209 kB 62.6 MB/s 
     |████████████████████████████████| 81 kB 11.5 MB/s 
     |████████████████████████████████| 78 kB 9.2 MB/s 
     |████████████████████████████████| 49 kB 8.4 MB/s 
     |████████████████████████████████| 147 kB 66.6 MB/s 
     |████████████████████████████████| 112 kB 35.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ff97763653417815ede39f5e1dcd89f7e65fdf15010e49992b3672b4127706e4
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


#### dataset

In [ ]:
!pip install torchdata

# https://github.com/Sam-Armstrong/tinyGPT/blob/main/Training.py
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torchtext.datasets import WikiText2
from tqdm import tqdm
import logging


logger = logging.getLogger(__name__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class WikiDataset(Dataset):
    """Dataset functionality from https://github.com/karpathy/minGPT"""
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        vocab_size = len(chars)
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        chunk = self.data[idx:idx + self.block_size + 1]
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype = torch.long)
        y = torch.tensor(dix[1:], dtype = torch.long)
        return x, y


seq_len = 128
# Code for using a text corpus contained within a .txt file
# data = open('input.txt', 'r').read()
# data = list(data)

text_iter = WikiText2(split = 'train')
train_data = []
for i, sequence in enumerate(text_iter):
    train_data += list(sequence)
# print(len(train_data)) # 10780437

# train_dataset = WikiDataset(train_data[:10000], seq_len) # one line of poem is roughly 50 characters
train_dataset = WikiDataset(train_data, seq_len) # one line of poem is roughly 50 characters
text_iter = WikiText2(split = 'test')
test_data = []
for i, sequence in enumerate(text_iter):
    test_data += list(sequence)
# print(len(data)) # 10780437
# test_dataset = WikiDataset(test_data[:1000], seq_len) # one line of poem is roughly 50 characters
test_dataset = WikiDataset(test_data, seq_len) # one line of poem is roughly 50 characters

vocab_size = train_dataset.vocab_size
# print("vocab_size",vocab_size) #283



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 10.5 MB/s 
     |████████████████████████████████| 139 kB 66.6 MB/s 
     |████████████████████████████████| 127 kB 67.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


#### functions

In [ ]:

def off_diagonal(x):
    # print("off_diagonal",x.shape)
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()


#### jepa

In [ ]:
# self.embedding = nn.Embedding(vocab_size, embed_dim)
# self.position_embedding = nn.Parameter(torch.zeros(1, seq_len, embed_dim))

# embedding = self.embedding(idx)
# position_embedding = self.position_embedding[:, :seq_len, :].repeat(batch_size, 1, 1)
# # Concats token and position and embeddings then projects them onto the embedding dimension
# x = torch.concat((embedding, position_embedding), dim = -1)


class JEPA(nn.Module):
    # def __init__(self, xin_channels, dim_sx, dim_sy, dim_z, dim_v, n_actions, space_dims, hidden_dims):
    def __init__(self, vocab_size, embed_size, in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v):
        super(JEPA, self).__init__()
        self.enc_x = nn.Sequential(
            nn.Embedding(vocab_size, embed_size),
            nn.Flatten(start_dim=1),
            nn.Linear(in_dimx*embed_size, dim_sx),
            )
        self.enc_y = nn.Sequential(
            nn.Embedding(vocab_size, embed_size),
            nn.Flatten(start_dim=1),
            nn.Linear(in_dimx*embed_size, dim_sx),
            )
        self.pred = nn.Sequential(
            nn.Linear(dim_sx + dim_z, dim_sy),
            nn.ReLU(True),
            )
        self.exp_x = nn.Sequential(
            nn.Linear(dim_sx, dim_v),
            nn.ReLU(True),
            )
        self.exp_y = nn.Sequential(
            nn.Linear(dim_sy, dim_v),
            nn.ReLU(True),
            )    

    def vicreg(self, x, y): # https://github.com/facebookresearch/vicreg/blob/main/main_vicreg.py
        # invariance loss
        repr_loss = F.mse_loss(x, y)
        x = x - x.mean(dim=0)
        y = y - y.mean(dim=0)

        # variance loss
        std_x = torch.sqrt(x.var(dim=0) + 0.0001)
        std_y = torch.sqrt(y.var(dim=0) + 0.0001)
        std_loss = torch.mean(F.relu(1 - std_x)) / 2 + torch.mean(F.relu(1 - std_y)) / 2

        # # covariance loss
        # cov_x = (x.T @ x) / (self.args.batch_size - 1)
        # cov_y = (y.T @ y) / (self.args.batch_size - 1)
        # cov_loss = off_diagonal(cov_x).pow_(2).sum().div(self.num_features)\
        #  + off_diagonal(cov_y).pow_(2).sum().div(self.num_features)

        # loss = (self.args.sim_coeff * repr_loss + self.args.std_coeff * std_loss + self.args.cov_coeff * cov_loss)

        batch_size=3
        num_features=3
        sim_coeff=1
        std_coeff=1
        cov_coeff=1

        # print("x.dim()",x.dim())
        if x.dim() == 1:
            x = x.view(-1, 1)
        
        if y.dim() == 1:
            y = y.view(-1, 1)
        x=x.T
        y=y.T
        # print("x",x.shape)
        cov_x = (x.T @ x) / (batch_size - 1)
        cov_y = (y.T @ y) / (batch_size - 1)
        # print("cov_x",cov_x.shape)
        cov_loss = off_diagonal(cov_x).pow_(2).sum().div(num_features)\
         + off_diagonal(cov_y).pow_(2).sum().div(num_features)

        loss = (sim_coeff * repr_loss + std_coeff * std_loss + cov_coeff * cov_loss)
        return loss

    # def argm(self, sx, sy):
    def argm(self, SX, SY):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        sampler = optuna.samplers.NSGAIISampler()
        # sampler = optuna.samplers.MOTPESampler()
        pruner = optuna.pruners.MedianPruner()
        batch_size=1
        # if sx.dim() == 2: batch_size,_=sx.shape
        if SX.dim() == 2: batch_size,_=SX.shape
        s=[]
        for i in range(batch_size):
            sx, sy = SX[i],SY[i]
            study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
            # study = optuna.create_study()
            def objective(trial):
                z = trial.suggest_uniform('z', -1, 1)
                # print("z trail",sx,z)
                # z=torch.tensor([z]).to(device)
                z=z.view(1,-1).to(device)
                sxz = torch.cat([sx, z], dim=-1)
                sy_ = self.pred(sxz)
                mseloss = nn.MSELoss()(sy, sy_)
                return mseloss
            study.optimize(objective, n_trials=10)
            st=study.best_params
            # print("st",st['z'])
            st=torch.tensor([st['z']])
            s.append(st)
        return torch.tensor(s)

    def loss(self, x, y):
        if x.dim()==2: batch_size,_=x.shape
        # print("loss",x,x.shape,x.dtype)
        sx = self.enc_x(x)
        sy = self.enc_y(y)
        sx=sx.flatten(start_dim=1)
        sy=sy.flatten(start_dim=1)
        # print("sx, sy",sx.shape, sy.shape) #10000
        z = self.argm(sx, sy).to(device)
        # z=np.array(z)
        z=torch.tensor(z).view(-1,1)
        # print("sx, z",sx.device, z.device) #[500, 20] [1]
        sxz = torch.cat([sx, z], dim=-1)
        sy_ = self.pred(sxz)
        # loss(sy, sy_)
        mseloss = nn.MSELoss()(sy, sy_)

        vx = self.exp_x(sx)
        vy = self.exp_y(sy)
        # print("vx",vx.shape) #[40]
        vicloss = self.vicreg(vx, vy)
        return mseloss + vicloss

    def forward(self, sx, a):
        # sx = self.enc_x(x)
        # sx=sx.flatten()
        sxz = torch.cat([sx, a], dim=-1)
        sy_ = self.pred(sxz)
        return sy_

in_dimx=128 # embedding size
in_dimy=128
dim_sx=20
dim_sy=20
dim_z=1
dim_v=40
# vocab_size=283
embed_size=4

# model = JEPA(in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v)
model = JEPA(vocab_size, embed_size, in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v)


# x=torch.rand(3, 210, 160)
# print("x",x.dtype) #

# y=torch.rand(3, 210, 160)
# x=torch.rand(in_dimx)
# y=torch.rand(in_dimy)
x=torch.randint(0,vocab_size,(500, 128), dtype = torch.long)
y=torch.randint(0,vocab_size,(500, 128), dtype = torch.long)
# print(x)
inv_loss = model.loss(x,y)
print(inv_loss)


# # # lin=nn.Linear(dim_sx + dim_z, dim_sy)
# # lin=nn.Embedding(283, dim_sx)
# lin=nn.Embedding(vocab_size, embed_size)
# now=nn.Linear(in_dimx*embed_size, dim_sx)
# # now=nn.Linear((in_dimx+dim_z)*embed_size, dim_sx)

# x=torch.randint(0,128,(500, 21), dtype = torch.long)
# print("x",x.dtype) #torch.float64
# print("x",x.shape,x.dtype) #
# x=lin(x)
# print("x",x.shape,x.dtype) #
# x=x.flatten(start_dim=1)
# print("x",x.shape,x.dtype) #
# x=now(x)
# print("x",x.shape,x.dtype) #

# x=torch.rand(500, 21)
# print("x",x.shape) #
# lop=nn.Linear(dim_sx + dim_z, dim_sy)
# x=lop(x)
# print("x",x.shape) #








# enc_x = nn.Sequential( # embed pi (240, 256, 3) -> 256 when flattened
#             nn.Conv2d(xin_channels, 8, 3, stride=2, padding=1), nn.ELU(),
#             # nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(8, 16, 5, stride=2, padding=2), nn.ELU(),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(16, 8, 7, stride=2, padding=3), nn.ELU(),
#             nn.Conv2d(8, 1, 5, stride=2, padding=2), nn.ELU(),
#             nn.AdaptiveAvgPool2d((5,4)),
#             # # nn.Conv2d(in_channels, out_channels=1, kernel_size=3, stride=2, padding=1),
#             # nn.ReLU(),
#             )
# x=torch.rand(3, 210, 160)
# sx=enc_x(x)
# print("sx.shape",sx.shape) # [1, 256] [1, 5, 4]
# sx=sx.flatten()
# print("sx.shape",sx.shape) # [20]


# pred = nn.Sequential(
#             nn.Linear(dim_sx + dim_z, dim_sy),
#             nn.ReLU(True),
#             )

# # sx =torch.rand(1, 16, 16)
# z =torch.rand(1)
# z=torch.tensor(z)
# sxz = torch.cat([sx, z], dim=-1)
# print(sxz.shape) #257
# sy_ = pred(sxz)


grad_clip_norm=1
lr=1e-4
betas=(0.9, 0.95)
batch_size=500


[W 2022-08-03 05:39:18,944] Trial 0 failed because of the following error: RuntimeError('Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_cat)')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-25-4aee1e424150>", line 97, in objective
    sxz = torch.cat([sx, z], dim=-1)
RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_cat)


RuntimeError: ignored

#### data train eval

In [ ]:

def train(loader, model, loss_fn, optimizer):
    model.train()
    losses = []
    pbar = tqdm(enumerate(loader), total = len(loader))
    for it, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)
        # print("x,y",x.shape,y.shape)
        # print("x,y",x.dtype,y.dtype) #torch.int64
        # print("x",x)
        with torch.set_grad_enabled(True):
            # logits = model(x)
            # # loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            # loss = loss_fn(logits, y)
            loss = model.loss(x,y)
            losses.append(loss.item())
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip_norm)
        optimizer.step()
        # lr = lr
        # pbar.set_description(f"epoch {epoch + 1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
        pbar.set_description(f"epoch {epoch + 1} iter {it}: train loss {loss.item():.5f}")


def eval(loader, model, loss_fn):
    model.eval()
    losses = []
    pbar = enumerate(loader)
    for it, (x, y) in pbar:
        x = x.to(device)
        y = y.to(device)
        with torch.set_grad_enabled(False):
            # logits = model(x)
            # # loss = criterion(logits.view(-1, logits.size(-1)), y.view(-1))
            # loss = loss_fn(logits, y)
            loss = model.loss(x,y)
            losses.append(loss.item())
    test_loss = float(np.mean(losses))
    logger.info("test loss: %f", test_loss)
    return test_loss


#### wwwwwwwwwww

In [ ]:

# in_dimx=128 # embedding size
# in_dimy=128
# dim_sx=20
# dim_sy=20
# dim_z=1
# dim_v=40
# # vocab_size=283
# embed_size=4
# model = JEPA(in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v)
model = JEPA(vocab_size, embed_size, in_dimx, in_dimy, dim_sx, dim_sy, dim_z, dim_v).to(device)


optimizer = torch.optim.AdamW(model.parameters(), lr = lr, betas = betas)
criterion = nn.CrossEntropyLoss()

def loss_fn(logits, y):
    return criterion(logits.view(-1, logits.size(-1)), y.view(-1))



In [ ]:
train_loader = DataLoader(train_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 4)
test_loader = DataLoader(test_dataset, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)

epochs=1
for epoch in range(epochs):
    # run_epoch(train_loader)
    train(train_loader, model, loss_fn, optimizer)
    if test is not None:
        test_loss = eval(test_loader, model, loss_fn)
        print('Test Loss:', test_loss)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
epoch 1 iter 30: train loss 2186.90161:   0%|          | 31/21561 [01:42<19:48:27,  3.31s/it]


KeyboardInterrupt: ignored

#### inference

In [ ]:

def inference(model, x, max_steps = 512, seq_len=seq_len):
    # seq_len = model.seq_len
    # seq_len = seq_len
    model.eval()
    print("test",x)
    for n in range(max_steps):
        if x.shape[1] <= seq_len:
            x_bar = x
        else:
            x_bar = x[:, -seq_len:]
        # print("test",x_bar)
        # output = model(x_bar)
        a=torch.zeros(1,1)
        print(x_bar.shape, a.shape)
        output = model(x_bar, a)
        # print("output",output)
        output = output[:, -1, :]
        output = F.softmax(output, dim = -1)
        ix = torch.multinomial(output, num_samples = 1)
        x = torch.cat((x, ix), dim = 1)
    return x

context = "This is what "
#context = 'There are many things about horses that have been discovered in recent'
# print([train_dataset.stoi[s] for s in context])
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype = torch.long)[None,...].to(device)
y = inference(model, x)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
print(completion)



In [ ]:
embed_dim = 512

Embedding = nn.Embedding(vocab_size, embed_dim)
position_embedding = torch.zeros(1, seq_len, embed_dim)
lin=nn.Linear(embed_dim * 2, embed_dim)


context = "This is what "
#context = 'There are many things about horses that have been discovered in recent'
# print([train_dataset.stoi[s] for s in context])
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype = torch.long)[None,...].to(device)
max_steps = 20#512
x=x.cpu()
# model.eval()
print("test",x)
for n in range(max_steps):
    if x.shape[1] <= seq_len:
        x_bar = x
    else:
        x_bar = x[:, -seq_len:]
    # print("test",x_bar)
    # output = model(x_bar)
    a=torch.zeros(1,1)
    print(x_bar.shape, a.shape)
    # output = mod(x_bar)

    idx=x_bar
    batch_size = idx.shape[0] # 1
    seq_len = idx.shape[1] # len(contex)+
    print("Embedding",Embedding)
    embedding = Embedding(idx) # [1, len(contex)+, embed_size]
    position_embedding = position_embedding[:, :seq_len, :].repeat(batch_size, 1, 1)
    # Concats token and position and embeddings then projects them onto the embedding dimension
    x = torch.concat((embedding, position_embedding), dim = -1)
    print("wp.s",embedding.shape, position_embedding.shape)
    # output=x[0]
    x=lin(x)
    print(x.shape)
    output=x
    # # output = mod(x_bar, a)
    # print("test",x_bar.shape) # [1, len(contex)+] int to 277+
    # output = model(x_bar)
    # print("output",output.shape) # [1, len(contex)+, vocab_size=283] float
    output = output[:, -1, :] #get logit for last character
    output = F.softmax(output, dim = -1) #vocab_size to char
    ix = torch.multinomial(output, num_samples = 1)
    x = torch.cat((x, ix), dim = 1)

# y = inference(model, x)[0]


test tensor([[53, 72, 73, 83,  1, 73, 83,  1, 87, 72, 65, 84,  1]])
torch.Size([1, 13]) torch.Size([1, 1])
Embedding Embedding(283, 512)
wp.s torch.Size([1, 13, 512]) torch.Size([1, 13, 512])
torch.Size([1, 13, 512])


RuntimeError: ignored